# Capstone - Toronto Shelter Occupancy Prediction Data Cleaning & Feature Engineering

---

## Import Modules

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
import requests
import json
import dateutil.parser
import matplotlib.pyplot as plt
from astral import Astral

# Ignore warnings
warnings.filterwarnings("ignore")

---

## Data Clearning & Feature Engineering

### Shelter Occupancy Data

In [2]:
# Getting occupancy data
df_17 = pd.read_csv('data/daily-shelter-occupancy-2017-csv.csv')
df_18 = pd.read_csv('data/daily-shelter-occupancy-2018-csv.csv')
df_19 = pd.read_csv('data/daily-shelter-occupancy-2019-csv.csv')
df = pd.concat([df_17, df_18, df_19])

In [3]:
# Dropping unuseful columns
df.drop(['_id', 'id'], axis=1, inplace=True)

In [4]:
# Checking if there are any rows with null or zero value in CAPACITY but non-zero value in OCCUPANCY
for program in df[((df.CAPACITY.isna()) | (df.CAPACITY == 0)) & (df.OCCUPANCY != 0)]['PROGRAM_NAME'].value_counts().index:
    # Assuming the max OCCUPANCY as the CAPACITY
    df.loc[df.PROGRAM_NAME == program, 'CAPACITY'] = df[df.PROGRAM_NAME == program].OCCUPANCY.max()

In [5]:
# Checking if there are any rows with null or zero values in OCCUPANCY, and drop these rows
df.drop(df[(df.OCCUPANCY == 0) | (df.OCCUPANCY.isna())].index, inplace=True)

In [6]:
# Calculating occupancy rate
df['OCCUPANCY_RATE'] = round(df.OCCUPANCY / df.CAPACITY, 4)

# Changing data type of CAPACITY column
df.CAPACITY = df.CAPACITY.astype(int)

# Changing data type of OCCUPANCY_DATE
df.OCCUPANCY_DATE = pd.to_datetime(df.OCCUPANCY_DATE)

In [7]:
# Getting addresses with empty postal codes
missing_postal_code_addresses = df[df.SHELTER_POSTAL_CODE.isna()].SHELTER_ADDRESS.value_counts().index

# Getting correct postal codes from Google Maps
postal_dict = {
    '38 Bathrust St': 'M5V 3W3',
    '67 Adelaide Street East' : 'M5C 1K6', 
    '1673 Kingston Road' : 'M1N 1S6',
    '1651 Sheppard Ave West' : 'M3M 2X4', 
    '129 Peter St' : 'M5V 1X1'
}

# Filling missing postal codes
for address in missing_postal_code_addresses:
    df.loc[df.SHELTER_ADDRESS == address, 'SHELTER_POSTAL_CODE'] = postal_dict[address]

In [8]:
# Cleaning postal code format

# Remove non-alphanumeric characters
df.SHELTER_POSTAL_CODE = df.SHELTER_POSTAL_CODE.replace('[^a-zA-Z0-9]', '' ,regex=True)

# Separate two components with space
df.SHELTER_POSTAL_CODE = df.SHELTER_POSTAL_CODE.apply(lambda s: s[:3] + ' ' + s[3:])

# Cleaning wrong postal code characters (M2N OE3)
df.loc[df.SHELTER_POSTAL_CODE == 'M2N OE3', 'SHELTER_POSTAL_CODE'] = 'M2N 0E3'

*Since this project focuses on the shelter occupancy data related homelessness, a judgement call is needed to exclude programs related to new immigrant/refugee settlement.* 

In [9]:
# df[df.PROGRAM_NAME.str.contains('Refugee')].PROGRAM_NAME.unique()
# df[df.PROGRAM_NAME.str.contains('Hotel')].PROGRAM_NAME.unique()
# df[df.PROGRAM_NAME.str.contains('Welcome')].PROGRAM_NAME.unique()
# df[df.PROGRAM_NAME.str.contains('Settlement')].PROGRAM_NAME.unique()

# df[df.FACILITY_NAME.str.contains('Refugee')].FACILITY_NAME.unique()
# df[df.FACILITY_NAME.str.contains('Hotel')].FACILITY_NAME.unique()
# df[df.FACILITY_NAME.str.contains('Welcome')].FACILITY_NAME.unique()
# df[df.FACILITY_NAME.str.contains('Settlement')].FACILITY_NAME.unique()

# df.ORGANIZATION_NAME.unique()

In [10]:
# Deciding programs and facilities to be excluded from the data
program_names_to_exclude = [
    'COSTI Reception Ctr CITY Program'
    'Christie Refugee Welcome Ctr - Settlement and Support',
    'Birkdale Residence  - Single Male Refugee Program',
    'COSTI - Quality Suites Singles Refugee Program',
    'Christe Refugee Welcome Centre - Singles',
    'Birkdale Residence  - Toronto Plaza Hotel Program (Refugees)',
    'COSTI Edward Hotel Refugee Family',
    'COSTI Edward Hotel Refugee Singles',
    'Sojourn House - Refugee Family Hotel Program-Grange',
    'COSTI - Toronto Plaza Hotel (Refugees)'
]

facility_names_to_exclude = [
    'Christie Refugee Welcome Centre',
    'HFS - Refugee Response Centre -'
]

orgnization_names_to_exclude = [
    'COSTI Immigrant Services',
    'Christie Refugee Welcome Centre, Inc.'
]

# Excluding irrelavent data
df = df.drop(df[df.PROGRAM_NAME.isin(program_names_to_exclude)].index)
df = df.drop(df[df.FACILITY_NAME.isin(facility_names_to_exclude)].index)
df = df.drop(df[df.ORGANIZATION_NAME.isin(orgnization_names_to_exclude)].index)

In [11]:
# Writing shelter occupancy data to a csv file
df.to_csv('data/shelter_occupancy_cleaned.csv')

---

### Weather Data

In [12]:
# Gettting weather data
weather_2017_df = pd.read_csv('data/2017-toronto-weather.csv')
weather_2018_df = pd.read_csv('data/2018-toronto-weather.csv')
weather_2019_df = pd.read_csv('data/2019-toronto-weather.csv')
weather_df = pd.concat([weather_2017_df, weather_2018_df, weather_2019_df])

In [13]:
# Cleaning weather data, dropping unnecessary columns
weather_df = weather_df.drop(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID', 'Data Quality',
       'Max Temp Flag', 'Min Temp Flag', 'Mean Temp Flag', 'Heat Deg Days Flag', 'Cool Deg Days Flag',
       'Total Rain Flag', 'Total Snow Flag', 'Total Precip Flag', 'Snow on Grnd (cm)',
       'Snow on Grnd Flag', 'Dir of Max Gust (10s deg)', 'Dir of Max Gust Flag', 'Spd of Max Gust (km/h)',
       'Spd of Max Gust Flag','Total Rain (mm)', 'Total Snow (cm)', 
        'Heat Deg Days (°C)', 'Cool Deg Days (°C)'], axis=1)

In [14]:
# Calculating the lengths of daylight each day in hours

# Coordinates of Toronto
latitude = 43.7001100
longitude = -79.4163000

# Function to calculate daylight hours in Toronto on a specific day
def get_daylight_hours(year, month, day):
    astral = Astral()
    sr, ss = astral.daylight_utc(datetime.date(year,month,day), latitude, longitude)
    return (ss - sr).seconds / 3600

weather_df['Daylight_Hours'] = weather_df.apply(lambda df : get_daylight_hours(df.Year, df.Month, df.Day), axis=1)

In [15]:
# Getting the day of week of each day
weather_df['Week_Day_Int'] = weather_df.apply(lambda df : datetime.date(df.Year, df.Month, df.Day).weekday(), axis=1)

In [16]:
# Converting days of week to literal format
def get_week_day(week_day_int):
    week_day_dict = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']
    return week_day_dict[week_day_int]

weather_df['Week_Day'] = weather_df.apply(lambda df: get_week_day(df.Week_Day_Int), axis=1)

In [17]:
# Convert date in weather data frame to datetime
weather_df['Date/Time'] = pd.to_datetime(weather_df['Date/Time'])

In [18]:
# Dropping unnecessary columns after daylight hours calculations
weather_df = weather_df.drop(['Year', 'Month', 'Day', 'Max Temp (°C)', 'Min Temp (°C)'], axis=1)

In [19]:
# Changing date time column name to match shelter occupancy data column name before merging
weather_df = weather_df.rename(columns={'Date/Time': 'OCCUPANCY_DATE'})

In [20]:
# Writing weather data to a csv file
weather_df.to_csv('data/toronto_weather.csv')

---